In [2]:
# General Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from selenium import webdriver
import main
import seaborn as sns
import chromedriver_autoinstaller
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
from collections import defaultdict
import pickle
from PIL import Image
import urllib
from adjustText import adjust_text


In [3]:
chromedriver_autoinstaller.install()

import warnings
warnings.filterwarnings('ignore')

## Pull Data

In [4]:
chromedriver_autoinstaller.install()

if __name__ == "__main__":
    driver = webdriver.Chrome()
    
# whoscored match centre url of the required match (Example: Barcelona vs Sevilla)
url = "https://www.whoscored.com/Matches/1640902/Live/England-Premier-League-2022-2023-Chelsea-Fulham"
match_data = main.getMatchData(driver, url, close_window=True)

# Match dataframe containing info about the match
matches_df = main.createMatchesDF(match_data)

# Events dataframe      
events_df = main.createEventsDF(match_data)

# match Id
matchId = match_data['matchId']

# Information about respective teams as dictionary
home_data = matches_df['home'][matchId]
away_data = matches_df['away'][matchId]
score = matches_df['score'][matchId]
score = ' ' + score.replace(':', '-') + ' '

home_name = home_data['name']
away_name = away_data['name']
date = match_data['startDate'].split('T')[0]
date_ = datetime.strptime(date, '%Y-%m-%d').date()
date_str = date_.strftime('%d/%m/%Y')

Region: England, League: Premier League, Season: 2022/2023, Match Id: 1640902


In [24]:
homeID = home_data['teamId']
awayID = away_data['teamId']

names = {
    homeID : home_name, 
    awayID : away_name
}

## Colours 

In [23]:
home_color = '#2D5DE7'
away_color = '#9C9C9C'

## xThreat via Pass
- Using Karun Singh's values

In [7]:
zones = []
for i in range(8):
    row = []
    for j in range(12):
        zone = i + ((j)*8)
        row.append(zone)
    zones.append(row)
zones = np.array(zones)
zones

array([[ 0,  8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88],
       [ 1,  9, 17, 25, 33, 41, 49, 57, 65, 73, 81, 89],
       [ 2, 10, 18, 26, 34, 42, 50, 58, 66, 74, 82, 90],
       [ 3, 11, 19, 27, 35, 43, 51, 59, 67, 75, 83, 91],
       [ 4, 12, 20, 28, 36, 44, 52, 60, 68, 76, 84, 92],
       [ 5, 13, 21, 29, 37, 45, 53, 61, 69, 77, 85, 93],
       [ 6, 14, 22, 30, 38, 46, 54, 62, 70, 78, 86, 94],
       [ 7, 15, 23, 31, 39, 47, 55, 63, 71, 79, 87, 95]])

In [8]:
xT_list = []
path = "https://karun.in/blog/data/open_xt_12x8_v1.json"
xT = pd.read_json(path)
xT = np.array(xT).T
xT_rows, xT_cols = xT.shape

x_bins = np.array(np.linspace(0,100, 13), dtype=object)
y_bins = np.array(np.linspace(0, 100, 9), dtype=object)

In [14]:
# Home Team
df_passes = events_df[(events_df['type'] == 'Pass') & 
                      (events_df['outcomeType'] == 'Successful') & 
                      (events_df['teamId'] == homeID) & 
                      (events_df['passFreekick'] == False) & 
                      (events_df['passCorner'] == False) &
                      (events_df['throwIn'] == False)].reset_index(drop=True)

df_passes['x1_bin'] = pd.cut(df_passes['x'], bins=x_bins, labels=False, include_lowest=True)
df_passes['x2_bin'] = pd.cut(df_passes['endX'], bins=x_bins, labels=False, include_lowest=True)
df_passes['y1_bin'] = pd.cut(df_passes['y'], bins=y_bins, labels=False, include_lowest=True)
df_passes['y2_bin'] = pd.cut(df_passes['endY'], bins=y_bins, labels=False, include_lowest=True)
df_passes['xT_start'] = df_passes[['x1_bin', 'y1_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
df_passes['xT_end'] = df_passes[['x2_bin', 'y2_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
df_passes['xT_diff'] = df_passes['xT_end'] - df_passes['xT_start']
df_passes['zone'] = df_passes[['x1_bin', 'y2_bin']].apply(lambda x:zones[x[1]][x[0]], axis=1)
df_passes = df_passes[df_passes['xT_diff'] > 0].reset_index(drop=True)
df_passes = df_passes[['playerName', 'xT_diff']]
df_passes_home = (df_passes.groupby('playerName')['xT_diff']
                           .sum()
                           .reset_index()
)
df_passes_home.rename({'xT_diff':'xT_Pass'}, axis='columns', inplace=True)
df_passes_home


# Away Team
df_passes = events_df[(events_df['type'] == 'Pass') & 
                      (events_df['outcomeType'] == 'Successful') & 
                      (events_df['teamId'] == awayID) & 
                      (events_df['passFreekick'] == False) & 
                      (events_df['passCorner'] == False) &
                      (events_df['throwIn'] == False)].reset_index(drop=True)

df_passes['x1_bin'] = pd.cut(df_passes['x'], bins=x_bins, labels=False, include_lowest=True)
df_passes['x2_bin'] = pd.cut(df_passes['endX'], bins=x_bins, labels=False, include_lowest=True)
df_passes['y1_bin'] = pd.cut(df_passes['y'], bins=y_bins, labels=False, include_lowest=True)
df_passes['y2_bin'] = pd.cut(df_passes['endY'], bins=y_bins, labels=False, include_lowest=True)
df_passes['xT_start'] = df_passes[['x1_bin', 'y1_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
df_passes['xT_end'] = df_passes[['x2_bin', 'y2_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
df_passes['xT_diff'] = df_passes['xT_end'] - df_passes['xT_start']
df_passes['zone'] = df_passes[['x1_bin', 'y2_bin']].apply(lambda x:zones[x[1]][x[0]], axis=1)
df_passes = df_passes[df_passes['xT_diff'] > 0].reset_index(drop=True)
df_passes = df_passes[['playerName', 'xT_diff']]
df_passes_away = (df_passes.groupby('playerName')['xT_diff']
                           .sum()
                           .reset_index()
)
df_passes_away.rename({'xT_diff':'xT_Pass'}, axis='columns', inplace=True)
df_passes_away

,playerName,xT_Pass
0,Aleksandar Mitrovic,0.021656
1,Andreas Pereira,0.037148
2,Antonee Robinson,0.020227
3,Bernd Leno,0.078918
4,Bobby De Cordova-Reid,0.065709
5,Harrison Reed,0.079463
6,Harry Wilson,0.009982
7,Issa Diop,0.038193
8,João Palhinha,0.031513
9,Kenny Tete,0.346676


## xThreat via Carry

In [15]:
df_carries = pd.DataFrame()
min_len = 3
max_len = 60
min_time = 1
max_time = 12

for idx, match_event in events_df.iterrows():
    if idx == len(events_df) - 1:
        break
    
    cur_team = match_event['teamId']
    idx_next = idx + 1
    events_df_next = events_df.loc[idx_next]
    take_ons = 0
    loop = True
    count = 0 
    
    while loop:
        next_events = events_df.loc[idx_next]
        
        if next_events['type'] == 'TakeOn' and next_events['outcomeType'] == 'Successful':
            take_ons += 1
            loop = True
        
        elif (next_events['type'] == 'TakeOn' and next_events['outcomeType'] == 'Unsuccessful') or (next_events['teamId'] != cur_team and next_events['type'] == 'Challenge' and next_events['outcomeType'] == 'Unsuccessful') or (next_events['type'] == 'Foul'):
            loop = True
            
        else:
            loop = False
            
        idx_next += 1
        count += 0 
        

    same_team = cur_team == next_events['teamId']
    not_ball_touch = match_event['type'] != 'BallTouch'
    dx = 105 * (match_event['endX'] - next_events['x'])/100
    dy = 68 * (match_event['endY'] - next_events['y'])/100
    min_dist = dx**2 + dy**2 >= min_len**2
    max_dist = dx**2 + dy**2 <= max_len**2
    same_period = match_event['period'] == next_events['period']
    
    valid_carry = same_team & not_ball_touch & min_dist & max_dist & same_period 
    
    if valid_carry:
        carry = pd.DataFrame()
        prev = match_event
        nxt = next_events
        carry.loc[0, 'eventId'] = prev['eventId'] + count
        carry['id'] = prev['id'] + count
        carry['teamId'] = nxt['teamId']
        carry['x'] = prev['endX']
        carry['y'] = prev['endY']
        carry['period'] = nxt['period']
        carry['type'] = carry.apply(lambda x: {'value': 99, 'displayName': 'Carry'}, axis=1)
        carry['outcomeType'] = 'Successful'
        carry['qualifiers'] = carry.apply(
            lambda x: {'type': {'value': 999, 'displayName': 'takeOns'}, 'value': str(take_ons)}, axis=1)
        carry['satisfiedEventsTypes'] = carry.apply(lambda x: [], axis=1)
        carry['isTouch'] = True
        carry['playerId'] = nxt['playerId']
        carry['playerName'] = nxt['playerName']
        carry['endX'] = nxt['x']
        carry['endY'] = nxt['y']
        
        df_carries = pd.concat([df_carries, carry], ignore_index=True, sort=False)


In [19]:
# Home Team
df_carries_home = df_carries[df_carries['teamId'] == homeID]
df_carries_home['x1_bin'] = pd.cut(df_carries_home['x'], bins=x_bins, labels=False, include_lowest=True)
df_carries_home['x2_bin'] = pd.cut(df_carries_home['endX'], bins=x_bins, labels=False, include_lowest=True)
df_carries_home['y1_bin'] = pd.cut(df_carries_home['y'], bins=y_bins, labels=False, include_lowest=True)
df_carries_home['y2_bin'] = pd.cut(df_carries_home['endY'], bins=y_bins, labels=False, include_lowest=True)
df_carries_home['xT_start'] = df_carries_home[['x1_bin', 'y1_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
df_carries_home['xT_end'] = df_carries_home[['x2_bin', 'y2_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
df_carries_home['xT_diff'] = df_carries_home['xT_end'] - df_carries_home['xT_start']
df_carries_home['zone'] = df_carries_home[['x1_bin', 'y2_bin']].apply(lambda x:zones[x[1]][x[0]], axis=1)
df_carries_home = df_carries_home[df_carries_home['xT_diff'] > 0].reset_index(drop=True)
df_carries_home = df_carries_home[['playerName', 'xT_diff']]
df_carries_home = (df_carries_home.groupby('playerName')['xT_diff']
                                  .sum()
                                  .reset_index()
                  )
df_carries_home.rename({'xT_diff':'xT_Carry'}, axis='columns', inplace=True)
df_carries_home


# Away Team
df_carries_away = df_carries[df_carries['teamId'] == awayID]
df_carries_away['x1_bin'] = pd.cut(df_carries_away['x'], bins=x_bins, labels=False, include_lowest=True)
df_carries_away['x2_bin'] = pd.cut(df_carries_away['endX'], bins=x_bins, labels=False, include_lowest=True)
df_carries_away['y1_bin'] = pd.cut(df_carries_away['y'], bins=y_bins, labels=False, include_lowest=True)
df_carries_away['y2_bin'] = pd.cut(df_carries_away['endY'], bins=y_bins, labels=False, include_lowest=True)
df_carries_away['xT_start'] = df_carries_away[['x1_bin', 'y1_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
df_carries_away['xT_end'] = df_carries_away[['x2_bin', 'y2_bin']].apply(lambda x:xT[x[0]][x[1]], axis=1)
df_carries_away['xT_diff'] = df_carries_away['xT_end'] - df_carries_away['xT_start']
df_carries_away['zone'] = df_carries_away[['x1_bin', 'y2_bin']].apply(lambda x:zones[x[1]][x[0]], axis=1)
df_carries_away = df_carries_away[df_carries_away['xT_diff'] > 0].reset_index(drop=True)
df_carries_away = df_carries_away[['playerName', 'xT_diff']]
df_carries_away = (df_carries_away.groupby('playerName')['xT_diff']
                                  .sum()
                                  .reset_index()
                  )
df_carries_away.rename({'xT_diff':'xT_Carry'}, axis='columns', inplace=True)
df_carries_away

,playerName,xT_Carry
0,Aleksandar Mitrovic,0.015192
1,Andreas Pereira,0.013027
2,Antonee Robinson,0.009422
3,Bernd Leno,0.001417
4,Bobby De Cordova-Reid,0.029019
5,Harrison Reed,0.047411
6,Issa Diop,0.014587
7,João Palhinha,0.009079
8,Kenny Tete,0.035428
9,Manor Solomon,0.002794


## Merge Datasets

In [21]:
df_home = (df_passes_home.merge(df_carries_home, on='playerName', how='outer')
                         .fillna(0)
          )
df_home


df_away = (df_passes_away.merge(df_carries_away, on='playerName', how='outer')
                         .fillna(0)
          )
df_away

,playerName,xT_Pass,xT_Carry
0,Aleksandar Mitrovic,0.021656,0.015192
1,Andreas Pereira,0.037148,0.013027
2,Antonee Robinson,0.020227,0.009422
3,Bernd Leno,0.078918,0.001417
4,Bobby De Cordova-Reid,0.065709,0.029019
5,Harrison Reed,0.079463,0.047411
6,Harry Wilson,0.009982,0.000000
7,Issa Diop,0.038193,0.014587
8,João Palhinha,0.031513,0.009079
9,Kenny Tete,0.346676,0.035428


## Plot 

In [45]:
fig, ax = plt.subplots(figsize = (20,15))
fig.set_facecolor("#2B2B2B")
ax.patch.set_facecolor("#2B2B2B")
# mpl.rcParams['figure.dpi'] = 800


# Home Team
plt.scatter(df_home.xT_Carry, df_home.xT_Pass, color=home_color, s=40)

text = []
for i, row in df_home.iterrows():
    p1 = row['playerName']
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    x = row['xT_Carry']
    y = row['xT_Pass']
    sf = float(x+y)
    t = plt.text(x = x+0.0005, y = y+0.002, s = p1.upper(), 
                 bbox=dict(facecolor=home_color, edgecolor='none', boxstyle='round', alpha=0.4), 
                 color='white', fontname = 'Sans Serif', fontsize = max(12, min(50, 85*((2*x)+y))))
    text.append(t)
    

# Away Team
plt.scatter(df_away.xT_Carry, df_away.xT_Pass, color=away_color, s=40)

for i, row in df_away.iterrows():
    p1 = row['playerName']
    p1 = p1.replace('-', ' ')
    p1 = p1.split(' ')
    if len(p1) > 2:
        name = "".join([x[0].upper() for x in p1])
        p1 = name
    elif len(p1) == 1:
        p1 = p1[0]
    else:
        p1 = p1[1]
    x = row['xT_Carry']
    y = row['xT_Pass']
    sf = float(x+y)
    t = plt.text(x = x+0.0005, y = y+0.002, s = p1.upper(), 
                 bbox=dict(facecolor=away_color, edgecolor='none', boxstyle='round', alpha=0.4), 
                 color='white', fontname = 'Sans Serif', fontsize = max(12, min(50, 85*((2*x)+y))))
    text.append(t)
    
plt.title('\nExpected Threat \n\n'.upper(), 
          loc='left', color='w', fontname = 'Sans Serif', fontsize = 30, y=1)
plt.suptitle(t = home_name.upper() + ' ' +score + ' ' + away_name.upper() + '  |  ' + date_str.upper() + '  |  '  + 'CREATED BY ...', x=0.125, y=0.93, ha='left', 
             fontsize=24, color='w')
plt.xlabel('\nxT via Carry', color='w', fontsize=30)
plt.ylabel('xT via Pass\n', color='w', fontsize=30)

ax.set_axisbelow(True)
ax.yaxis.grid(color='#848484', linestyle='dashed')
ax.xaxis.grid(color='#848484', linestyle='dashed')
# spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_color('w')
ax.spines['bottom'].set_color('w')
# ticks
for tick in ax.xaxis.get_ticklines(): 
    tick.set_color('w')
for tick in ax.yaxis.get_ticklines(): 
    tick.set_color('w')
plt.xticks(color='w')
plt.yticks(color='w')
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

adjust_text(text, ax=ax)


fig.savefig('xT_scatter.png', 
            bbox_inches="tight",
            edgecolor="none",
            facecolor = "#2B2B2B",
            dpi=500)